In [1]:
import pandas as pd
audio_impro_processed = pd.read_csv('audio_df_improvised.csv')

audio_impro_processed.reset_index(inplace=True)
audio_impro_processed.head()

,index,start_time,end_time,wav_file,emotion,val,act,dom
0,0,6.2901,8.2357,Ses01F_impro01_F000,neu,2.5,2.5,2.5
1,1,10.0100,11.3925,Ses01F_impro01_F001,neu,2.5,2.5,2.5
2,2,14.8872,18.0175,Ses01F_impro01_F002,neu,2.5,2.5,2.5
3,3,27.4600,31.4900,Ses01F_impro01_F005,neu,2.5,3.5,2.0
4,4,85.2700,88.0200,Ses01F_impro01_F012,ang,2.0,3.5,3.5


In [2]:
import numpy as np
import librosa
def get_melspectrogram_db(file_path, med_duration ,sr=None, n_fft=2048, hop_length=256, n_mels=128, fmin=100, fmax=15000, top_db=80):
    med_duration = int(np.floor(med_duration))
    wav,sr = librosa.load(file_path,sr=sr)
    if wav.shape[0]<(med_duration*sr):
        wav=np.pad(wav,int(np.ceil(((med_duration*sr)-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:(med_duration*sr)]
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels)
#     librosa.decompose.nn_filter(spec, aggregate=np.median, metric='cosine')
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    return spec_db

In [3]:
def spec_to_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    return spec_scaled

In [4]:
# We split the dataframe into train and test segments

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

train, valid = train_test_split(audio_impro_processed, test_size=0.2, shuffle=False)
train = audio_impro_processed[:2212]
train = shuffle(train)
valid = audio_impro_processed[2212:]

In [5]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os

class IEMOCAP(Dataset):
    def __init__(self, base, df, in_col, out_col):
        self.df = df
        self.data = []
        self.labels = []
        self.c2i={}
        self.i2c={}
        self.categories = sorted(df[out_col].unique())
        print(self.categories)
        for i, category in enumerate(self.categories):
            self.c2i[category]=i
            self.i2c[i]=category
        for ind in tqdm(range(len(df))):
            row = df.iloc[ind]
            
            #If all files are stored in a folder named sentences
            file_path = base + '/' + row[in_col] + '.wav'
            
            
            self.data.append(spec_to_image(get_melspectrogram_db(file_path, 3))[np.newaxis,...])
            self.labels.append(self.c2i[row['emotion']])
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]




In [10]:
train_data = IEMOCAP('/data/home/advaitmb/datasets/sentences', train, 'wav_file', 'emotion')
valid_data = IEMOCAP('/data/home/advaitmb/datasets/sentences', valid, 'wav_file', 'emotion')
# train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=16)
# valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True, num_workers=16)

  0%|          | 9/2212 [00:00<00:24, 88.24it/s]

['ang', 'hap', 'neu', 'sad']


  1%|▏         | 10/731 [00:00<00:07, 95.33it/s]

['ang', 'hap', 'neu', 'sad']


100%|██████████| 731/731 [00:07<00:00, 92.65it/s]


In [6]:
import torchvision.models as models
model = models.resnet18(pretrained=True)

In [19]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
import torch.nn as nn
model.fc = nn.Linear(512, 1024)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [8]:
import torch
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')

model = model.to(device)

In [13]:
data = torch.from_numpy(np.array(train_data.data))
data.shape

torch.Size([2212, 1, 128, 188])

In [16]:
i = 0
for data in train_data.data:
    data = np.expand_dims(data, axis=1)
    data = torch.from_numpy(data)
    data = data.to(device, dtype=torch.float32)
    features[i] = model(data)
    i+=1

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 11.17 GiB total capacity; 10.81 GiB already allocated; 832.00 KiB free; 58.07 MiB cached)

In [15]:
features = torch.zeros((train.shape[0], 1024))



In [48]:
for i, data in enumerate(train_loader):
    x, y = data
    print(type(x))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'tor